In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
import sympy as sp
from itertools import product

from Init import init_S, init_R, init_G
from Groups import GroupOhD, NUM_ELEMENTS 
from Functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants

In [2]:
IRREPS  = {'2Oh' : ['G1g','G1u','Hg','Hu'],
           '2C4v': ['G1','G2'],
           '2C2v': ['G'],
           '2C3v': ['G','F1','F2']}
PREFS   = {'2Oh' : (0,0,0), 
           '2C4v': list(init_G['2C4v'].keys())[0], #0,0,1
           '2C2v': list(init_G['2C2v'].keys())[0], #0,1,1
           '2C3v': list(init_G['2C3v'].keys())[0],}#1,1,1

## Matrix representations

In [3]:
#initialize group
group_2Oh = GroupOhD()

#spin1212 matrix
S = extract_repr(group_2Oh,init_S)

#spin1 matrix
Rot3d = extract_repr(group_2Oh,init_R)

### Irrep matrix

In [4]:
#irreps 2Oh
irrep_2Oh  = { I : extract_repr(group_2Oh,init_G['2Oh'][I]) for I in IRREPS['2Oh']}

#irreps 2C4v
group_2C4v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C4v']))
irrep_2C4v = { I : extract_repr(group_2C4v,init_G['2C4v'][PREFS['2C4v']][I]) for I in IRREPS['2C4v']}

#irreps 2C3v
group_2C3v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C3v']))
irrep_2C3v = { I : extract_repr(group_2C3v,init_G['2C3v'][PREFS['2C3v']][I]) for I in IRREPS['2C3v']}

#irreps 2C2v
group_2C2v = group_2Oh.little_group(Rot3d,sp.Matrix(PREFS['2C2v']))
irrep_2C2v = { I : extract_repr(group_2C2v,init_G['2C2v'][PREFS['2C2v']][I]) for I in IRREPS['2C2v']}

In [31]:
irrepsG    = { '2Oh':irrep_2Oh, '2C4v':irrep_2C4v, '2C3v':irrep_2C3v, '2C2v':irrep_2C2v }
GROUPS     = { '2Oh':group_2Oh, '2C4v':group_2C4v, '2C3v':group_2C3v, '2C2v':group_2C2v }

## Print irrepsG

In [62]:
tofile={}
listR = sorted(NUM_ELEMENTS,key=lambda x:NUM_ELEMENTS[x])

#for each group
for G, Ilist in IRREPS.items():
    #for each irrep
    for I in Ilist:
        #first line
        tofile[(G,I)]=[r'# rot_mat_table group '+G+'\tirrep '+I+'\tptot '+'_'.join(map(str,PREFS[G]))+'\tsize'+str(irrepsG[G][I]['E'].shape)]
        iR=0
        #for element in double cover of cubic group (no I)
        for Rname in listR:
            #if element non in little group, skip
            if Rname not in GROUPS[G].list_elements():
                continue
            #number convention used in Marcus code. dictionary
            Rnum = NUM_ELEMENTS[Rname]
            irlab= 'R'
            # R $Rnum
            tofile[(G,I)].append('# [rot_mat_table] '+irlab+' %d'%(Rnum-1))
            #print each element
            for irow,row in enumerate(irrepsG[G][I][Rname].tolist()):
                for jcol, col in enumerate(row):
                    entry = complex(col)
                    re_e, im_e = entry.real, entry.imag
                    tofile[(G,I)].append("rot_mat_table_"+irlab+"[[%2d]][%d,%d] <-\t%.16e +\t%.16e*1.i"%(iR+1,irow+1,jcol+1,re_e,im_e))
            #append space for readibility
            tofile[(G,I)].append(" ")
            #increment counter 
            iR+=1
        iR=0
        
        #the same for I+R
        for Rname in listR:
            if 'I_s'+Rname not in GROUPS[G].list_elements():
                continue
            Rnum = NUM_ELEMENTS[Rname]
            irlab= 'IR'
            tofile[(G,I)].append('# [rot_mat_table] '+irlab+' %d'%(Rnum-1))
            for irow,row in enumerate(irrepsG[G][I]['I_s'+Rname].tolist()):
                for jcol, col in enumerate(row):
                    entry = complex(col)
                    re_e, im_e = entry.real, entry.imag
                    tofile[(G,I)].append("rot_mat_table_"+irlab+"[[%2d]][%d,%d] <-\t%.16e +\t%.16e*1.i"%(iR+1,irow+1,jcol+1,re_e,im_e))
            tofile[(G,I)].append(" ")
            iR+=1
            

In [63]:
for (G,I), filecontent in tofile.items():
    with open('rot_mat_table_'+G+'_'+I+'_p'+'_'.join(map(str,PREFS[G]))+'.txt','w') as writer:
        for line in tofile[(G,I)]:
            writer.write(line+'\n')